# Meta Kaggle and Meta Kaggle Code Exploratory Data Analysis

![Kaggle Logo](data/kaggle_data/kaggle-logo.png)

---

## 1.) <u>Accelerators, Imports, and Datasets' Size Check</u>

---

### 1.A) *GPU Acceleration*

In [1]:
# GPU acceleration for pandas and sklearn
%load_ext cudf.pandas
%load_ext cuml.accel

### 1.B) *Python Library Imports*

In [12]:
# general imports
from  pathlib import Path
from datetime import datetime, timedelta

# data science/scientific imports
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.stats import pearsonr
from scipy.sparse import csr_matrix, coo_matrix

# file conversion imports
import pyarrow as pa
import pyarrow.parquet as pq
from multiprocessing import Pool, cpu_count

# visualization imports
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go

# machine learning imports
from sklearn.cluster import KMeans
from sklearn.preprocessing import Normalizer, normalize, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA, TruncatedSVD, NMF
from sklearn.feature_extraction.text import TfidfVectorizer

# currency conversion import
from currency_converter import CurrencyConverter as cc

### 1.C) *Meta-Kaggle `.csv` Files' Size Check*
- prints files in order from smallest to largest file size

In [9]:
directory = Path('data/kaggle_data/meta-kaggle/')

sorted_files = sorted(
	(f for f in directory.iterdir() if f.is_file()),
    key=lambda x: x.stat().st_size
    )
for file in sorted_files:
    size_in_mb = file.stat().st_size / (1 << 20)
    print(f'{file.name}: {size_in_mb:.3f}MB')

KernelVotes.csv: 0.000MB
KernelAcceleratorTypes.csv: 0.000MB
KernelLanguages.csv: 0.000MB
CompetitionTags.csv: 0.027MB
Tags.csv: 0.096MB
UserOrganizations.csv: 0.148MB
Organizations.csv: 0.275MB
ModelTags.csv: 0.280MB
DatasetTaskSubmissions.csv: 0.622MB
ModelVotes.csv: 2.263MB
ModelVariations.csv: 2.862MB
DatasetTasks.csv: 2.894MB
Models.csv: 3.193MB
ForumMessageReactions.csv: 8.569MB
ModelVersions.csv: 11.474MB
DatasetTags.csv: 17.163MB
KernelVersionModelSources.csv: 17.786MB
ModelVariationVersions.csv: 20.929MB
KernelTags.csv: 27.628MB
Forums.csv: 30.192MB
Datasources.csv: 41.714MB
KernelVersionKernelSources.csv: 42.528MB
ForumTopics.csv: 64.760MB
UserFollowers.csv: 76.268MB
Datasets.csv: 83.611MB
DatasetVotes.csv: 101.145MB
Competitions.csv: 111.418MB
KernelVersionCompetitionSources.csv: 152.390MB
ForumMessageVotes.csv: 192.802MB
Kernels.csv: 260.587MB
KernelVersionDatasetSources.csv: 369.191MB
TeamMemberships.csv: 379.493MB
Teams.csv: 684.027MB
DatasetVersions.csv: 1124.922MB
Forum

---

## 2.) <u>Data Loading</u>

---

### 2.A) *Data Loading Strategy*

**Strategy**: All CSV files used in this notebook are converted to **Parquet** format once (stored in `data/kaggle_data/meta-kaggle-parquet/`).<br><br>
Parquet is a columnar binary format that provides:
- **~10-20x faster reads** — no CSV parsing overhead
- **~60-80% smaller on disk** — built-in compression
- **Free column selection** — `columns=` only reads selected columns from disk<br><br>

After the one-time conversion, every loading cell runs in seconds rather than minutes.

| Size Category | Files | Approach |
|---------------|---|---|
| Small (<1MB)  | KernelLanguages, AcceleratorTypes, Tags, CompetitionTags | Direct CSV (fast enough) |
| Medium–Huge   | All others | Read from parquet with `columns=` |

### 2.B) *Reference Tables (from `.csv`) File Loading*

In [11]:
# base path
META_KAGGLE = Path('data/kaggle_data/meta-kaggle/')

# ── tiny reference tables ──
kernel_languages = pd.read_csv(META_KAGGLE / 'KernelLanguages.csv')
accelerator_types = pd.read_csv(META_KAGGLE / 'KernelAcceleratorTypes.csv')
tags = pd.read_csv(META_KAGGLE / 'Tags.csv')
comp_tags = pd.read_csv(META_KAGGLE / 'CompetitionTags.csv')

print("=== Reference Tables ===")
for name, df in [('KernelLanguages', kernel_languages), ('AcceleratorTypes', accelerator_types),
                 ('Tags', tags), ('CompetitionTags', comp_tags)]:
    print(f"  {name}: {df.shape}  |  Memory: {df.memory_usage(deep=True).sum() / 1e6:.4f} MB")

=== Reference Tables ===
  KernelLanguages: (10, 4)  |  Memory: 0.0004 MB
  AcceleratorTypes: (14, 2)  |  Memory: 0.0003 MB
  Tags: (831, 9)  |  Memory: 0.1071 MB
  CompetitionTags: (1201, 3)  |  Memory: 0.0304 MB


### 2.C) *One-Time CSV → Parquet Conversion*

- Run the cell below once to convert all large CSVs to parquet
- It skips files that already exist, so re-running is safe
- Very large files (>200 MB) are streamed in chunks to limit memory usage during conversion

In [ ]:
# ── Storage Configuration ──
PARQUET_DIR = Path('data/kaggle_data/meta-kaggle-parquet/')
RANDOM_STATE = 33

In [ ]:
PARQUET_DIR.mkdir(parents=True, exist_ok=True)

# CSVs used in this notebook
_CONVERT = [
    'Competitions', 'Kernels', 'KernelVersionCompetitionSources', 'KernelTags',
    'Teams', 'TeamMemberships', 'KernelVersions', 'Users', 'Submissions',
    'UserAchievements',
]
_CHUNK_THRESHOLD_MB = 2000


def _convert_csv_to_parquet(name):
    """Convert a single CSV to parquet. Returns (name, status_message)."""
    parquet_path = PARQUET_DIR / f'{name}.parquet'
    csv_path = META_KAGGLE / f'{name}.csv'

    if parquet_path.exists():
        pq_mb = parquet_path.stat().st_size / (1 << 20)
        return (name, f"{pq_mb:.3f} MB — already exists, skipping")

    csv_mb = csv_path.stat().st_size / (1 << 20)

    if csv_mb > _CHUNK_THRESHOLD_MB:
        # Stream chunks to parquet to limit peak memory
        writer = None
        for chunk in pd.read_csv(csv_path, chunksize=1000000):
            table = pa.Table.from_pandas(chunk, preserve_index=False)
            if writer is None:
                writer = pq.ParquetWriter(parquet_path, table.schema)
            writer.write_table(table)
        if writer:
            writer.close()
    else:
        pd.read_csv(csv_path).to_parquet(parquet_path, index=False)

    pq_mb = parquet_path.stat().st_size / (1 << 20)
    return (name, f"{csv_mb:.0f} MB CSV -> {pq_mb:.3f} MB parquet ({pq_mb / csv_mb * 100:.3f}%)")


# Use 5 workers to keep disk I/O saturated without thrashing
num_workers = min(5, cpu_count())
print(f"Converting with {num_workers} parallel workers...\n")

with Pool(num_workers) as pool:
    results = pool.map(_convert_csv_to_parquet, _CONVERT)

for name, msg in results:
    print(f"  {name}: {msg}")

print("\nConversion complete.")

### 2.D) *Medium Tables (from `.parquet`) File Loading*

In [ ]:
# ── medium tables (from parquet) ──
competitions = pd.read_parquet(PARQUET_DIR / 'Competitions.parquet', columns=[
    'Id', 'Title', 'EnabledDate', 'DeadlineDate', 'MaxTeamSize',
    'RewardType', 'RewardQuantity', 'NumPrizes', 'TotalTeams',
    'TotalCompetitors', 'TotalSubmissions', 'HostName', 'OnlyAllowKernelSubmissions'])
competitions['EnabledDate'] = pd.to_datetime(competitions['EnabledDate'], format='mixed', errors='coerce')
competitions['DeadlineDate'] = pd.to_datetime(competitions['DeadlineDate'], format='mixed', errors='coerce')

kernels = pd.read_parquet(PARQUET_DIR / 'Kernels.parquet', columns=[
    'Id', 'AuthorUserId', 'CurrentKernelVersionId', 'CreationDate', 'Medal', 'TotalVotes', 'TotalViews'])
kernels['CreationDate'] = pd.to_datetime(kernels['CreationDate'], format='mixed', errors='coerce')

kv_comp_sources = pd.read_parquet(PARQUET_DIR / 'KernelVersionCompetitionSources.parquet')
kernel_tags = pd.read_parquet(PARQUET_DIR / 'KernelTags.parquet')

print("=== Medium Tables ===")
for name, df in [('Competitions', competitions), ('Kernels', kernels),
                 ('KVCompSources', kv_comp_sources), ('KernelTags', kernel_tags)]:
    print(f"  {name}: {df.shape}  |  Memory: {df.memory_usage(deep=True).sum() / 1e6:.1f} MB")

### 2.E) *Large Tables (from `.parquet`) File Loading*

In [ ]:
# ── large tables (from parquet) ──
teams = pd.read_parquet(PARQUET_DIR / 'Teams.parquet', columns=[
    'Id', 'CompetitionId', 'TeamLeaderId', 'TeamName', 'Medal',
    'PublicLeaderboardRank', 'PrivateLeaderboardRank'])

team_memberships = pd.read_parquet(PARQUET_DIR / 'TeamMemberships.parquet')

kernel_versions = pd.read_parquet(PARQUET_DIR / 'KernelVersions.parquet', columns=[
    'Id', 'ScriptId', 'ScriptLanguageId', 'AuthorUserId', 'CreationDate',
    'AcceleratorTypeId', 'TotalLines', 'RunningTimeInMilliseconds'])
kernel_versions['CreationDate'] = pd.to_datetime(
    kernel_versions['CreationDate'], format='mixed', errors='coerce')

users = pd.read_parquet(PARQUET_DIR / 'Users.parquet', columns=[
    'Id', 'UserName', 'DisplayName', 'RegisterDate', 'PerformanceTier'])
users['RegisterDate'] = pd.to_datetime(users['RegisterDate'], format='mixed', errors='coerce')

submissions = pd.read_parquet(PARQUET_DIR / 'Submissions.parquet', columns=[
    'Id', 'SubmittedUserId', 'TeamId', 'SourceKernelVersionId', 'SubmissionDate'])
submissions['SubmissionDate'] = pd.to_datetime(
    submissions['SubmissionDate'], format='mixed', errors='coerce')

print("Large table summary:")
for name, df in [('Teams', teams), ('TeamMemberships', team_memberships),
                 ('KernelVersions', kernel_versions), ('Users', users),
                 ('Submissions', submissions)]:
    print(f"  {name}: {df.shape}  |  Memory: {df.memory_usage(deep=True).sum() / 1e6:.1f} MB")

### 2.F) *Extra-Large Tables (from `.parquet`) File Loading*

In [ ]:
# ── UserAchievements (from parquet, filter to Competitions only) ──
# No date filter — achievement data is cumulative and has no date column.
user_achievements = pd.read_parquet(
    PARQUET_DIR / 'UserAchievements.parquet',
    columns=['UserId', 'AchievementType', 'Tier',
             'TotalGold', 'TotalSilver', 'TotalBronze',
             'Points', 'HighestRanking'])

_before = len(user_achievements)
user_achievements = user_achievements[
    user_achievements['AchievementType'] == 'Competitions'].reset_index(drop=True)

print(f"UserAchievements: {_before:,} -> {len(user_achievements):,} rows (Competitions only)")
print(f"  Memory: {user_achievements.memory_usage(deep=True).sum() / 1e6:.1f} MB")

#### Data Loading — Quick Schema Reference

| DataFrame | Rows (approx) | Key Columns | Joins To |
|---|---------------|---|---|
| `competitions` | ~1K           | Id, EnabledDate, RewardType, MaxTeamSize | Teams(CompetitionId), CompetitionTags(CompetitionId) |
| `teams` | ~millions     | Id, CompetitionId, TeamLeaderId, Medal, Ranks | TeamMemberships(TeamId), Submissions(TeamId), Users(TeamLeaderId) |
| `team_memberships` | ~millions     | TeamId, UserId | Teams(Id), Users(Id) |
| `kernel_versions` | ~40+ millions | Id, ScriptId, ScriptLanguageId, AcceleratorTypeId | KernelLanguages, AcceleratorTypes, KVCompSources |
| `users` | ~20+ millions | Id, PerformanceTier, RegisterDate | Teams, Kernels, Submissions |
| `submissions` | ~20+ millions | TeamId, SourceKernelVersionId | Teams, KernelVersions |
| `user_achievements` | ~80+ millions | UserId, Tier, TotalGold/Silver/Bronze | Users(Id) |

---

## 3.) <u>Meta-Kaggle Exploratory Data Analysis (EDA)</u>

---

### 3.A) *Competition Landscape Overview*

Before diving into the specific research questions, let's understand the overall Kaggle competition ecosystem:
- how many competitions have been launched over time
- what kinds of rewards they offer
- how large they are
- what subject areas they cover

#### 3.A.i) ***Competitions Per Year***

In [ ]:
# competitions launched per year
comp_by_year = competitions.copy()
comp_by_year['Year'] = comp_by_year['EnabledDate'].dt.year
yearly_comps = comp_by_year.groupby('Year').size().reset_index(name='Count')
yearly_comps = yearly_comps[yearly_comps['Year'].between(2010, 2025)]

fig = px.bar(yearly_comps, x='Year', y='Count',
             title='Number of Kaggle Competitions Launched Per Year',
             labels={'Count': 'Number of Competitions'},
             color_discrete_sequence=['#20BEFF'])
fig.update_layout(width=1000, height=500, title_x=0.5)
fig.show()

#### 3.A.ii) ***Competition Reward Types***

In [ ]:
# reward type distribution
reward_counts = competitions['RewardType'].value_counts().reset_index()
reward_counts.columns = ['RewardType', 'Count']

fig = px.pie(reward_counts, names='RewardType', values='Count',
             title='Distribution of Competition Reward Types',
             color_discrete_sequence=px.colors.qualitative.Set2)
fig.update_layout(width=800, height=500, title_x=0.5)
fig.show()

#### 3.A.iii) ***Competition Cash Reward Sizes (USD)***

In [ ]:
# Filter to cash prize competitions and convert to USD
converter = cc()

# Get competitions with cash rewards (filter out Knowledge, Jobs, etc.)
cash_comps = competitions[
    (competitions['RewardType'].str.contains('USD|EUR|GBP|INR|Prize', case=False, na=False)) &
    (competitions['RewardQuantity'].notna()) &
    (competitions['RewardQuantity'] > 0)
].copy()

# Extract currency and amount from RewardType
# RewardType format examples: "USD $100,000", "EUR €50,000", "Prize"
def extract_currency_and_amount(row):
    """Extract currency code and convert reward to USD."""
    reward_type = str(row['RewardType'])
    amount = row['RewardQuantity']
    
    # Default to USD if not specified or if it's just "Prize"
    if 'EUR' in reward_type or '€' in reward_type:
        currency = 'EUR'
    elif 'GBP' in reward_type or '£' in reward_type:
        currency = 'GBP'
    elif 'INR' in reward_type or '₹' in reward_type:
        currency = 'INR'
    else:
        currency = 'USD'  # Default assumption
    
    # Convert to USD
    try:
        if currency == 'USD':
            return amount
        else:
            return converter.convert(amount, currency, 'USD')
    except:
        # If conversion fails, assume it's already USD
        return amount

cash_comps['RewardUSD'] = cash_comps.apply(extract_currency_and_amount, axis=1)

# Create histogram
fig = px.histogram(
    cash_comps,
    x='RewardUSD',
    nbins=40,
    title='Distribution of Competition Cash Rewards (USD)',
    labels={'RewardUSD': 'Total Prize Pool (USD)'},
    color_discrete_sequence=['#20BEFF']
)
fig.update_layout(
    width=1000,
    height=500,
    title_x=0.5,
    xaxis_title='Total Prize Pool (USD)',
    yaxis_title='Number of Competitions',
    xaxis_tickformat='$,.0f'
)
fig.show()

print(f"\nCash Prize Competition Statistics (n={len(cash_comps):,}):")
print(cash_comps['RewardUSD'].describe().apply(lambda x: f'${x:,.0f}'))

#### 3.A.iv) ***Competition Sizes (Total Competitors)***

In [ ]:
# competition size distribution
fig = px.histogram(competitions, x='TotalCompetitors',
                   nbins=50,
                   title='Distribution of Competition Sizes (by Total Competitors)',
                   labels={'TotalCompetitors': 'Total Competitors'},
                   color_discrete_sequence=['#20BEFF'])
fig.update_layout(width=1000, height=500, title_x=0.5,
                  xaxis_title='Total Competitors',
                  yaxis_title='Number of Competitions')
fig.show()

print("Competition Size Summary Statistics:")
print(competitions[['TotalTeams', 'TotalCompetitors', 'TotalSubmissions']].describe().round(1))

#### 3.A.v) ***Competition Cash Reward (USD) vs. Total Competitors***

In [ ]:
# Scatter plot: Total Prize Pool (USD) vs. Total Competitors
# Filter to competitions with both cash rewards and competitor data
scatter_data = cash_comps[
    (cash_comps['TotalCompetitors'].notna()) &
    (cash_comps['TotalCompetitors'] > 0)
].copy()

# Add reward tier categories for color coding
def categorize_reward(amount):
    if amount >= 1_000_000:
        return '$1M+'
    elif amount >= 100_000:
        return '$100K-$1M'
    elif amount >= 10_000:
        return '$10K-$100K'
    else:
        return '<$10K'

scatter_data['RewardTier'] = scatter_data['RewardUSD'].apply(categorize_reward)

# Create scatter plot with log scale for better visualization
fig = px.scatter(
    scatter_data,
    x='RewardUSD',
    y='TotalCompetitors',
    color='RewardTier',
    title='Competition Prize Pool (USD) vs. Total Competitors',
    labels={
        'RewardUSD': 'Total Prize Pool (USD)',
        'TotalCompetitors': 'Total Competitors',
        'RewardTier': 'Prize Tier'
    },
    color_discrete_sequence=px.colors.qualitative.Bold,
    category_orders={'RewardTier': ['<$10K', '$10K-$100K', '$100K-$1M', '$1M+']},
    hover_data={
        'Title': True,
        'RewardUSD': ':$,.0f',
        'TotalCompetitors': ':,',
        'RewardTier': False
    }
)

# Use log scale for both axes to better show the relationship
fig.update_xaxes(type='log', tickformat='$,.0f')
fig.update_yaxes(type='log', tickformat=',')

fig.update_layout(
    width=1000,
    height=600,
    title_x=0.5,
    hovermode='closest'
)

fig.show()

# Calculate correlation
from scipy.stats import spearmanr
corr, p_value = spearmanr(scatter_data['RewardUSD'], scatter_data['TotalCompetitors'])
print(f"\nSpearman Correlation (Prize Pool vs. Competitors): {corr:.3f} (p={p_value:.4f})")
print(f"Number of cash prize competitions with competitor data: {len(scatter_data):,}")

#### 3.A.vi) ***Competition Categories (via Tags)***

In [ ]:
# top competition categories via tags
comp_tag_names = comp_tags.merge(
    tags[['Id', 'Name', 'FullPath']], left_on='TagId', right_on='Id', suffixes=('', '_tag'))
top_comp_tags = comp_tag_names['Name'].value_counts().head(20).reset_index()
top_comp_tags.columns = ['Tag', 'Count']

fig = px.bar(top_comp_tags, x='Count', y='Tag', orientation='h',
             title='Top 20 Competition Tags',
             color_discrete_sequence=['#FF6F61'])
fig.update_layout(width=1000, height=600, title_x=0.5,
                  yaxis={'categoryorder': 'total ascending'})
fig.show()

#### 3.A.vii) ***Medal Rate by Competition Size — Are Smaller Competitions More Winnable?***

In [ ]:
# Medal rate by competition size
teams_comp = teams.merge(
    competitions[['Id', 'TotalCompetitors']],
    left_on='CompetitionId', right_on='Id', suffixes=('', '_comp'))
teams_comp = teams_comp[teams_comp['TotalCompetitors'].notna()].copy()

teams_comp['SizeBin'] = pd.cut(
    teams_comp['TotalCompetitors'],
    bins=[0, 100, 500, 1000, 2000, 5000, 100_000],
    labels=['<100', '100-500', '500-1K', '1K-2K', '2K-5K', '5K+'])
teams_comp['HasMedal'] = teams_comp['Medal'].notna() & (teams_comp['Medal'] != '')

rate_by_size = teams_comp.groupby('SizeBin', observed=True).agg(
    TotalTeams=('HasMedal', 'count'),
    MedalTeams=('HasMedal', 'sum')
).reset_index()
rate_by_size['MedalRate'] = (rate_by_size['MedalTeams'] / rate_by_size['TotalTeams'] * 100).round(2)

fig = px.bar(rate_by_size, x='SizeBin', y='MedalRate', text='MedalRate',
             title='Medal Rate by Competition Size (% of Teams Earning Any Medal)',
             labels={'SizeBin': 'Competition Size (Total Competitors)', 'MedalRate': 'Medal Rate (%)'},
             color_discrete_sequence=['#20BEFF'])
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(width=1000, height=500, title_x=0.5)
fig.show()

print("Medal rate by competition size:")
print(rate_by_size.to_string(index=False))

### 3.B) Coding Languages & Hardware (Research Questions 1 & 5)

> *What coding languages and code libraries are most often used?*
> *What types of hardware are being used, and do winners most often use professional/corporate level hardware or is average enthusiast hardware enough?*

We'll look at:
- Overall language distribution across all Kaggle notebooks/scripts
- Language trends over time
- Language distribution for **competition-linked** kernels specifically
- Hardware accelerator (GPU/TPU) usage and trends
- Accelerator usage in competition kernels

#### 3.B.i) ***Overall Coding-Language Distribution***

In [ ]:
# overall language distribution across ALL kernel versions
lang_dist = kernel_versions.merge(
    kernel_languages, left_on='ScriptLanguageId', right_on='Id', suffixes=('', '_lang'))
lang_counts = lang_dist['DisplayName'].value_counts().reset_index()
lang_counts.columns = ['Language', 'Count']

fig = px.pie(lang_counts, names='Language', values='Count',
             title='Programming Language Distribution — All Kaggle Notebooks & Scripts',
             color_discrete_sequence=px.colors.qualitative.Bold)
fig.update_layout(width=800, height=500, title_x=0.5)
fig.show()

print("Language counts:")
print(lang_counts.to_string(index=False))

#### 3.B.ii) ***Coding-Language Usage (over time)***

In [ ]:
# language usage trends over time
lang_dist['Year'] = lang_dist['CreationDate'].dt.year
lang_yearly = lang_dist.groupby(['Year', 'DisplayName']).size().reset_index(name='Count')
lang_yearly = lang_yearly[lang_yearly['Year'].between(2015, 2025)]

fig = px.line(lang_yearly, x='Year', y='Count', color='DisplayName',
              title='Programming Language Usage Over Time',
              labels={'Count': 'Number of Kernel Versions', 'DisplayName': 'Language'},
              markers=True)
fig.update_layout(width=1000, height=500, title_x=0.5)
fig.show()

#### 3.B.iii) ***Coding-Language Distribution for Competition-Linked Kernels***

In [ ]:
# language distribution for COMPETITION-LINKED kernels only
comp_kv_ids = set(kv_comp_sources['KernelVersionId'].unique())
comp_kernel_versions = kernel_versions[kernel_versions['Id'].isin(comp_kv_ids)]

print(f"Total kernel versions: {len(kernel_versions):,}")
print(f"Competition-linked kernel versions: {len(comp_kernel_versions):,}")
print(f"Percentage linked to competitions: {len(comp_kernel_versions)/len(kernel_versions)*100:.2f}%")

comp_lang = comp_kernel_versions.merge(
    kernel_languages, left_on='ScriptLanguageId', right_on='Id', suffixes=('', '_lang'))
comp_lang_counts = comp_lang['DisplayName'].value_counts().reset_index()
comp_lang_counts.columns = ['Language', 'Count']

fig = px.pie(comp_lang_counts, names='Language', values='Count',
             title='Language Distribution — Competition-Linked Kernels Only',
             color_discrete_sequence=px.colors.qualitative.Bold)
fig.update_layout(width=800, height=500, title_x=0.5)
fig.show()

print("\nCompetition-linked kernel language counts:")
print(comp_lang_counts.to_string(index=False))

#### 3.B.iv) ***Hardware Accelerator Usage***

In [ ]:
# hardware accelerator usage — all kernels
accel_dist = kernel_versions.merge(
    accelerator_types, left_on='AcceleratorTypeId', right_on='Id', suffixes=('', '_accel'))
accel_counts = accel_dist['Label'].value_counts().reset_index()
accel_counts.columns = ['Accelerator', 'Count']

fig = px.bar(accel_counts, x='Count', y='Accelerator', orientation='h',
             title='Hardware Accelerator Usage Across All Kernels',
             color_discrete_sequence=['#6C5B7B'])
fig.update_layout(width=1000, height=400, title_x=0.5,
                  yaxis={'categoryorder': 'total ascending'})
fig.show()

# percentage breakdown
accel_counts['Pct'] = (accel_counts['Count'] / accel_counts['Count'].sum() * 100).round(2)
print("Accelerator breakdown:")
print(accel_counts.to_string(index=False))

#### 3.B.v) ***Hardware Accelerator Adoption (over time; all kernels)***

In [ ]:
# GPU/TPU accelerator adoption over time (excluding "None")
accel_dist['Year'] = accel_dist['CreationDate'].dt.year
accel_yearly = (accel_dist[accel_dist['Label'] != 'None']
                .groupby(['Year', 'Label']).size()
                .reset_index(name='Count'))
accel_yearly = accel_yearly[accel_yearly['Year'].between(2015, 2025)]

fig = px.line(accel_yearly, x='Year', y='Count', color='Label',
              title='GPU / TPU Accelerator Adoption Over Time (Excluding "None")',
              labels={'Count': 'Number of Kernel Versions', 'Label': 'Accelerator'},
              markers=True)
fig.update_layout(width=1000, height=500, title_x=0.5)
fig.show()

#### 3.B.iv) ***Hardware Accelerator Usage for Competition-Linked Kernels***

In [ ]:
# accelerator usage in competition-linked kernels specifically
comp_accel = comp_kernel_versions.merge(
    accelerator_types, left_on='AcceleratorTypeId', right_on='Id', suffixes=('', '_accel'))
comp_accel_counts = comp_accel['Label'].value_counts().reset_index()
comp_accel_counts.columns = ['Accelerator', 'Count']
comp_accel_counts['Pct'] = (comp_accel_counts['Count'] / comp_accel_counts['Count'].sum() * 100).round(2)

fig = px.pie(comp_accel_counts, names='Accelerator', values='Count',
             title='Accelerator Usage in Competition-Linked Kernels',
             color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_layout(width=800, height=500, title_x=0.5)
fig.show()

print("Competition kernel accelerator breakdown:")
print(comp_accel_counts.to_string(index=False))

#### 3.B.vii) ***Compute Intensity vs. Competition Performance — Does Hardware Decide Winners?***

Do medal-winning competition submissions require longer runtimes, more code, or more powerful accelerators than non-winning ones? If not, clever feature engineering may matter more than raw compute.

In [ ]:
# Link competition-linked kernel versions to team medal outcomes via submissions
kernel_subs = submissions[submissions['SourceKernelVersionId'].notna()][
    ['SourceKernelVersionId', 'TeamId']].drop_duplicates()

comp_kv_outcomes = comp_kernel_versions.merge(
    kernel_subs, left_on='Id', right_on='SourceKernelVersionId', how='inner')
comp_kv_outcomes = comp_kv_outcomes.merge(
    teams[['Id', 'Medal']].rename(columns={'Id': 'TeamId_teams'}),
    left_on='TeamId', right_on='TeamId_teams', how='left')

comp_kv_outcomes['Outcome'] = comp_kv_outcomes['Medal'].apply(
    lambda x: 'Medal Winner' if pd.notna(x) and str(x).strip() not in ('', 'nan') else 'No Medal')

# Filter to kernels with valid runtime and code length
valid_kv = comp_kv_outcomes[
    (comp_kv_outcomes['RunningTimeInMilliseconds'] > 0) &
    (comp_kv_outcomes['TotalLines'] > 0)].copy()
valid_kv['RuntimeMinutes'] = valid_kv['RunningTimeInMilliseconds'] / 60_000

# Runtime comparison
fig = px.box(valid_kv, x='Outcome', y='RuntimeMinutes', color='Outcome',
             title='Kernel Runtime: Medal Winners vs. Non-Medal (Competition Submissions)',
             labels={'RuntimeMinutes': 'Runtime (minutes)', 'Outcome': ''},
             color_discrete_map={'Medal Winner': '#FFD700', 'No Medal': '#B0BEC5'})
fig.update_yaxes(type='log', title_text='Runtime (minutes, log scale)')
fig.update_layout(width=1000, height=500, title_x=0.5, showlegend=False)
fig.show()

# Code length comparison
fig = px.box(valid_kv, x='Outcome', y='TotalLines', color='Outcome',
             title='Code Length: Medal Winners vs. Non-Medal (Competition Submissions)',
             labels={'TotalLines': 'Total Lines of Code', 'Outcome': ''},
             color_discrete_map={'Medal Winner': '#FFD700', 'No Medal': '#B0BEC5'})
fig.update_yaxes(type='log', title_text='Total Lines of Code (log scale)')
fig.update_layout(width=1000, height=500, title_x=0.5, showlegend=False)
fig.show()

# Summary statistics
print(f"Kernels analyzed: {len(valid_kv):,}\n")
for outcome in ['Medal Winner', 'No Medal']:
    subset = valid_kv[valid_kv['Outcome'] == outcome]
    print(f"  {outcome} (n={len(subset):,}):")
    print(f"    Runtime — median: {subset['RuntimeMinutes'].median():.1f} min, "
          f"mean: {subset['RuntimeMinutes'].mean():.1f} min")
    print(f"    Code    — median: {subset['TotalLines'].median():.0f} lines, "
          f"mean: {subset['TotalLines'].mean():.0f} lines")

In [ ]:
# Accelerator usage: medal winners vs. non-medal in competition submissions
accel_outcomes = comp_kv_outcomes.merge(
    accelerator_types, left_on='AcceleratorTypeId', right_on='Id', suffixes=('', '_accel'))

# Compute percentage within each outcome group
accel_pcts = accel_outcomes.groupby(['Outcome', 'Label']).size().reset_index(name='Count')
totals = accel_pcts.groupby('Outcome')['Count'].transform('sum')
accel_pcts['Pct'] = (accel_pcts['Count'] / totals * 100).round(2)

fig = px.bar(accel_pcts, x='Label', y='Pct', color='Outcome',
             barmode='group',
             title='Accelerator Usage: Medal Winners vs. Non-Medal (Competition Submissions)',
             labels={'Label': 'Accelerator', 'Pct': 'Percentage within Group (%)'},
             color_discrete_map={'Medal Winner': '#FFD700', 'No Medal': '#B0BEC5'})
fig.update_layout(width=1000, height=500, title_x=0.5,
                  xaxis={'categoryorder': 'total descending'})
fig.show()

### 3.C) *Individuals vs. Teams (Research Question 3)*

> *Are competition winners most often individuals or teams?*

- We compute team sizes by counting memberships per team, then analyze how solo competitors compare to multi-member teams in terms of participation rates and medal outcomes.

#### 3.C.i)

In [ ]:
# compute team sizes
team_sizes = team_memberships.groupby('TeamId').size().reset_index(name='TeamSize')
teams_with_size = teams.merge(team_sizes, left_on='Id', right_on='TeamId', how='left')
teams_with_size['TeamSize'] = teams_with_size['TeamSize'].fillna(1).astype(int)
teams_with_size['IsSolo'] = teams_with_size['TeamSize'] == 1

print(f"Total teams: {len(teams_with_size):,}")
print(f"Solo competitors: {teams_with_size['IsSolo'].sum():,} ({teams_with_size['IsSolo'].mean()*100:.1f}%)")
print(f"Multi-member teams: {(~teams_with_size['IsSolo']).sum():,} ({(~teams_with_size['IsSolo']).mean()*100:.1f}%)")
print(f"\nTeam size statistics:")
print(teams_with_size['TeamSize'].describe())

In [ ]:
# team size distribution (capped at 20 for readability)
size_counts = teams_with_size['TeamSize'].value_counts().sort_index().reset_index()
size_counts.columns = ['TeamSize', 'Count']
size_counts = size_counts[size_counts['TeamSize'] <= 20]

fig = px.bar(size_counts, x='TeamSize', y='Count',
             title='Distribution of Team Sizes Across All Competitions',
             labels={'TeamSize': 'Number of Team Members', 'Count': 'Number of Teams'},
             color_discrete_sequence=['#F67280'])
fig.update_layout(width=1000, height=500, title_x=0.5)
fig.show()

In [ ]:
# medal distribution: solo vs team by size category
teams_with_medal = teams_with_size[teams_with_size['Medal'].notna() & (teams_with_size['Medal'] != '')]
teams_with_medal = teams_with_medal.copy()
teams_with_medal['SizeCategory'] = pd.cut(
    teams_with_medal['TeamSize'],
    bins=[0, 1, 2, 3, 5, 100],
    labels=['Solo', 'Duo', 'Trio', '4-5', '6+'])

medal_labels = {'1': 'Gold', '2': 'Silver', '3': 'Bronze'}
teams_with_medal['MedalName'] = teams_with_medal['Medal'].astype(str).map(medal_labels)

medal_by_size = teams_with_medal.groupby(['SizeCategory', 'MedalName']).size().reset_index(name='Count')

fig = px.bar(medal_by_size, x='SizeCategory', y='Count', color='MedalName',
             title='Medal Distribution by Team Size Category',
             labels={'SizeCategory': 'Team Size', 'Count': 'Number of Medals'},
             color_discrete_map={'Gold': '#FFD700', 'Silver': '#C0C0C0', 'Bronze': '#CD7F32'},
             barmode='group',
             category_orders={'MedalName': ['Gold', 'Silver', 'Bronze']})
fig.update_layout(width=1000, height=500, title_x=0.5)
fig.show()

In [ ]:
# medal RATE by team size — what proportion of teams earn any medal?
medal_rate = teams_with_size.copy()
medal_rate['HasMedal'] = medal_rate['Medal'].notna() & (medal_rate['Medal'] != '')
medal_rate['SizeCategory'] = pd.cut(
    medal_rate['TeamSize'],
    bins=[0, 1, 2, 3, 5, 100],
    labels=['Solo', 'Duo', 'Trio', '4-5', '6+'])

rate_by_size = medal_rate.groupby('SizeCategory', observed=True).agg(
    TotalTeams=('HasMedal', 'count'),
    MedalTeams=('HasMedal', 'sum')
).reset_index()
rate_by_size['MedalRate'] = (rate_by_size['MedalTeams'] / rate_by_size['TotalTeams'] * 100).round(2)

fig = px.bar(rate_by_size, x='SizeCategory', y='MedalRate',
             title='Medal Rate by Team Size (% of Teams Earning Any Medal)',
             labels={'SizeCategory': 'Team Size', 'MedalRate': 'Medal Rate (%)'},
             color_discrete_sequence=['#355C7D'],
             text='MedalRate')
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(width=1000, height=500, title_x=0.5)
fig.show()

In [ ]:
# solo vs team participation over time
teams_with_comp = teams_with_size.merge(
    competitions[['Id', 'EnabledDate']], left_on='CompetitionId', right_on='Id', suffixes=('', '_comp'))
teams_with_comp['Year'] = teams_with_comp['EnabledDate'].dt.year

yearly_solo = teams_with_comp.groupby('Year').agg(
    Total=('IsSolo', 'count'),
    SoloCount=('IsSolo', 'sum')
).reset_index()
yearly_solo['SoloPct'] = (yearly_solo['SoloCount'] / yearly_solo['Total'] * 100).round(1)
yearly_solo['TeamPct'] = (100 - yearly_solo['SoloPct']).round(1)
yearly_solo = yearly_solo[yearly_solo['Year'].between(2010, 2025)]

fig = go.Figure()
fig.add_trace(go.Bar(x=yearly_solo['Year'], y=yearly_solo['SoloPct'],
                     name='Solo', marker_color='#355C7D'))
fig.add_trace(go.Bar(x=yearly_solo['Year'], y=yearly_solo['TeamPct'],
                     name='Team', marker_color='#F67280'))
fig.update_layout(barmode='stack',
                  title='Solo vs. Team Participation Over Time',
                  xaxis_title='Year', yaxis_title='Percentage (%)',
                  width=1000, height=500, title_x=0.5)
fig.show()

### 3.D) Professional vs. Amateur Competitors (Research Question 4)

> *Are these winners professionals or amateurs in the fields of machine learning and data science?*

- Kaggle assigns each user a **Performance Tier** (Novice → Contributor → Expert → Master → Grandmaster)
- We use this as a proxy for experience level and examine how tier correlates with competition success.

In [ ]:
# performance tier distribution
tier_labels = {0: 'Novice', 1: 'Contributor', 2: 'Expert', 3: 'Master', 4: 'Grandmaster', 5: 'Staff'}
tier_order = ['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster', 'Staff']
users['TierLabel'] = users['PerformanceTier'].map(tier_labels)

tier_counts = users['TierLabel'].value_counts().reset_index()
tier_counts.columns = ['Tier', 'Count']

fig = px.bar(tier_counts, x='Tier', y='Count',
             title='Distribution of User Performance Tiers',
             color='Tier',
             color_discrete_map={
                 'Novice': '#B0BEC5', 'Contributor': '#81C784', 'Expert': '#4FC3F7',
                 'Master': '#BA68C8', 'Grandmaster': '#FFD54F', 'Staff': '#FF8A65'},
             category_orders={'Tier': tier_order})
fig.update_layout(width=1000, height=500, title_x=0.5, showlegend=False)
fig.show()

print("Performance Tier Counts:")
for tier in tier_order:
    c = tier_counts[tier_counts['Tier'] == tier]['Count'].values
    if len(c) > 0:
        print(f"  {tier:15s}: {c[0]:>10,}")

In [ ]:
# competition achievements by performance tier
ua_with_tier = user_achievements.merge(
    users[['Id', 'PerformanceTier', 'TierLabel']], left_on='UserId', right_on='Id', how='left')

comp_achievers = ua_with_tier['TierLabel'].value_counts().reset_index()
comp_achievers.columns = ['Tier', 'Count']

fig = px.pie(comp_achievers, names='Tier', values='Count',
             title='Performance Tier Distribution of Users with Competition Achievements',
             color='Tier',
             color_discrete_map={
                 'Novice': '#B0BEC5', 'Contributor': '#81C784', 'Expert': '#4FC3F7',
                 'Master': '#BA68C8', 'Grandmaster': '#FFD54F', 'Staff': '#FF8A65'})
fig.update_layout(width=800, height=500, title_x=0.5)
fig.show()

In [ ]:
# total competition medals (gold/silver/bronze) by performance tier
medal_by_tier = ua_with_tier.groupby('TierLabel').agg(
    TotalGold=('TotalGold', 'sum'),
    TotalSilver=('TotalSilver', 'sum'),
    TotalBronze=('TotalBronze', 'sum'),
    UserCount=('UserId', 'nunique')
).reset_index()

tier_rank_order = ['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster']
medal_by_tier = medal_by_tier[medal_by_tier['TierLabel'].isin(tier_rank_order)]
medal_by_tier['TierLabel'] = pd.Categorical(
    medal_by_tier['TierLabel'], categories=tier_rank_order, ordered=True)
medal_by_tier = medal_by_tier.sort_values('TierLabel')

fig = go.Figure()
fig.add_trace(go.Bar(x=medal_by_tier['TierLabel'], y=medal_by_tier['TotalGold'],
                     name='Gold', marker_color='#FFD700'))
fig.add_trace(go.Bar(x=medal_by_tier['TierLabel'], y=medal_by_tier['TotalSilver'],
                     name='Silver', marker_color='#C0C0C0'))
fig.add_trace(go.Bar(x=medal_by_tier['TierLabel'], y=medal_by_tier['TotalBronze'],
                     name='Bronze', marker_color='#CD7F32'))
fig.update_layout(barmode='group',
                  title='Total Competition Medals by Performance Tier',
                  xaxis_title='Performance Tier', yaxis_title='Total Medals',
                  width=1000, height=500, title_x=0.5)
fig.show()

In [ ]:
# average medals PER USER by tier (more meaningful than raw totals)
medal_by_tier['AvgGold'] = (medal_by_tier['TotalGold'] / medal_by_tier['UserCount']).round(3)
medal_by_tier['AvgSilver'] = (medal_by_tier['TotalSilver'] / medal_by_tier['UserCount']).round(3)
medal_by_tier['AvgBronze'] = (medal_by_tier['TotalBronze'] / medal_by_tier['UserCount']).round(3)

fig = go.Figure()
fig.add_trace(go.Bar(x=medal_by_tier['TierLabel'], y=medal_by_tier['AvgGold'],
                     name='Avg Gold', marker_color='#FFD700'))
fig.add_trace(go.Bar(x=medal_by_tier['TierLabel'], y=medal_by_tier['AvgSilver'],
                     name='Avg Silver', marker_color='#C0C0C0'))
fig.add_trace(go.Bar(x=medal_by_tier['TierLabel'], y=medal_by_tier['AvgBronze'],
                     name='Avg Bronze', marker_color='#CD7F32'))
fig.update_layout(barmode='group',
                  title='Average Competition Medals Per User by Performance Tier',
                  xaxis_title='Performance Tier', yaxis_title='Avg Medals per User',
                  width=1000, height=500, title_x=0.5)
fig.show()

print("Medals per user by tier:")
print(medal_by_tier[['TierLabel', 'UserCount', 'AvgGold', 'AvgSilver', 'AvgBronze']].to_string(index=False))

In [ ]:
# what performance tier are team leaders of MEDAL-WINNING teams?
medal_teams = teams_with_size[teams_with_size['Medal'].notna() & (teams_with_size['Medal'] != '')]
leader_tiers = medal_teams.merge(
    users[['Id', 'TierLabel']], left_on='TeamLeaderId', right_on='Id', how='left')

leader_tiers['MedalName'] = leader_tiers['Medal'].astype(str).map(
    {'1': 'Gold', '2': 'Silver', '3': 'Bronze'})
leader_tier_medal = leader_tiers.groupby(['TierLabel', 'MedalName']).size().reset_index(name='Count')

fig = px.bar(leader_tier_medal, x='TierLabel', y='Count', color='MedalName',
             title='Medal-Winning Team Leader Performance Tiers',
             labels={'TierLabel': 'Leader Performance Tier', 'Count': 'Medal-Winning Teams'},
             color_discrete_map={'Gold': '#FFD700', 'Silver': '#C0C0C0', 'Bronze': '#CD7F32'},
             barmode='group',
             category_orders={
                 'TierLabel': ['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster'],
                 'MedalName': ['Gold', 'Silver', 'Bronze']})
fig.update_layout(width=1000, height=500, title_x=0.5)
fig.show()

#### 3.D.ii) ***Competition Categories Where Amateurs Succeed***

Which competition tags (subject areas) give Novice and Contributor-tier users the best chance at earning a medal? This identifies the most accessible niches for newcomers.

In [ ]:
# Join team leaders to their performance tier and competition tags
team_leader_tier = teams.merge(
    users[['Id', 'PerformanceTier']], left_on='TeamLeaderId', right_on='Id', suffixes=('', '_user'))
team_leader_tier['HasMedal'] = team_leader_tier['Medal'].notna() & (team_leader_tier['Medal'] != '')
team_leader_tier['IsAmateur'] = team_leader_tier['PerformanceTier'].isin([0, 1])  # Novice, Contributor

# Get competition tag names
comp_tag_names = comp_tags.merge(tags[['Id', 'Name']], left_on='TagId', right_on='Id', suffixes=('', '_tag'))

# Join teams to competition tags
team_tags = team_leader_tier.merge(
    comp_tag_names[['CompetitionId', 'Name']], on='CompetitionId', how='inner')

# Amateur medal rate per tag (require >=100 amateur participants for significance)
amateur_by_tag = team_tags[team_tags['IsAmateur']].groupby('Name').agg(
    AmateurTeams=('HasMedal', 'count'),
    AmateurMedals=('HasMedal', 'sum')
).reset_index()
amateur_by_tag['MedalRate'] = (amateur_by_tag['AmateurMedals'] / amateur_by_tag['AmateurTeams'] * 100).round(2)
amateur_by_tag = amateur_by_tag[amateur_by_tag['AmateurTeams'] >= 100].sort_values(
    'MedalRate', ascending=False).head(20)

fig = px.bar(amateur_by_tag, x='MedalRate', y='Name', orientation='h',
             title='Top 20 Competition Categories by Amateur (Novice/Contributor) Medal Rate',
             labels={'Name': 'Competition Tag', 'MedalRate': 'Amateur Medal Rate (%)'},
             color='MedalRate',
             color_continuous_scale='YlOrRd',
             hover_data={'AmateurTeams': ':,', 'AmateurMedals': ':,'})
fig.update_layout(width=1000, height=600, title_x=0.5,
                  yaxis={'categoryorder': 'total ascending'},
                  coloraxis_showscale=False)
fig.show()

# Also show the comparison: amateur vs professional medal rate per tag
pro_by_tag = team_tags[~team_tags['IsAmateur']].groupby('Name').agg(
    ProTeams=('HasMedal', 'count'),
    ProMedals=('HasMedal', 'sum')
).reset_index()
pro_by_tag['ProMedalRate'] = (pro_by_tag['ProMedals'] / pro_by_tag['ProTeams'] * 100).round(2)

comparison = amateur_by_tag.merge(pro_by_tag[['Name', 'ProMedalRate']], on='Name', how='left')
comparison['AmateurAdvantageRatio'] = (comparison['MedalRate'] / comparison['ProMedalRate']).round(2)
comparison = comparison.sort_values('MedalRate', ascending=False)

print("Amateur vs. Professional medal rates (top 20 categories for amateurs):")
print(comparison[['Name', 'AmateurTeams', 'MedalRate', 'ProMedalRate', 'AmateurAdvantageRatio']]
      .rename(columns={'MedalRate': 'Amateur%', 'ProMedalRate': 'Pro%', 'AmateurAdvantageRatio': 'Ratio'})
      .to_string(index=False))

#### 3.D.iii) ***Profiling Gold Medals Won by Novice/Contributor-Tier Users***

Do amateurs ever win Gold? If so, what do those competitions look like — how large are they, what categories, are they kernel-only? This gives a concrete profile of "winnable" competitions.

In [ ]:
# Find Gold medal teams led by Novice or Contributor-tier users
tier_labels_map = {0: 'Novice', 1: 'Contributor', 2: 'Expert', 3: 'Master', 4: 'Grandmaster', 5: 'Staff'}

gold_teams = teams[teams['Medal'].notna() & (teams['Medal'] != '')].copy()
gold_teams['MedalNum'] = pd.to_numeric(gold_teams['Medal'], errors='coerce')
gold_teams = gold_teams[gold_teams['MedalNum'] == 1]  # Gold only

gold_leaders = gold_teams.merge(
    users[['Id', 'PerformanceTier', 'UserName']], left_on='TeamLeaderId', right_on='Id', suffixes=('', '_user'))
gold_leaders['TierLabel'] = gold_leaders['PerformanceTier'].map(tier_labels_map)

# Filter to amateur Gold winners (Novice + Contributor)
amateur_golds = gold_leaders[gold_leaders['PerformanceTier'].isin([0, 1])].copy()
amateur_golds = amateur_golds.merge(
    competitions[['Id', 'Title', 'TotalCompetitors', 'OnlyAllowKernelSubmissions', 'RewardType']],
    left_on='CompetitionId', right_on='Id', suffixes=('', '_comp'))

print(f"Total Gold medals across all competitions: {len(gold_leaders):,}")
print(f"Gold medals won by Novice/Contributor leaders: {len(amateur_golds):,} "
      f"({len(amateur_golds)/len(gold_leaders)*100:.1f}%)\n")

# Profile these competitions
print("=== Profile of Competitions Where Amateurs Won Gold ===")
print(f"  Median competition size: {amateur_golds['TotalCompetitors'].median():.0f} competitors")
print(f"  Mean competition size:   {amateur_golds['TotalCompetitors'].mean():.0f} competitors")
print(f"  Kernel-only: {amateur_golds['OnlyAllowKernelSubmissions'].sum()} / {len(amateur_golds)} "
      f"({amateur_golds['OnlyAllowKernelSubmissions'].mean()*100:.1f}%)")

# Competition size distribution for amateur Golds
fig = px.histogram(amateur_golds, x='TotalCompetitors', nbins=30,
                   title='Competition Size Distribution Where Amateurs (Novice/Contributor) Won Gold',
                   labels={'TotalCompetitors': 'Total Competitors'},
                   color_discrete_sequence=['#FFD700'])
fig.update_layout(width=1000, height=400, title_x=0.5)
fig.show()

# Tier breakdown of all Gold winners for context
gold_tier_counts = gold_leaders['TierLabel'].value_counts().reset_index()
gold_tier_counts.columns = ['Tier', 'GoldCount']

fig = px.pie(gold_tier_counts, names='Tier', values='GoldCount',
             title='Performance Tier of All Gold Medal Team Leaders',
             color='Tier',
             color_discrete_map={
                 'Novice': '#B0BEC5', 'Contributor': '#81C784', 'Expert': '#4FC3F7',
                 'Master': '#BA68C8', 'Grandmaster': '#FFD54F', 'Staff': '#FF8A65'})
fig.update_layout(width=800, height=500, title_x=0.5)
fig.show()

### 3.E) Cross-Cutting Analysis & Synthesis

Combining findings across the research questions to identify broader patterns:
- what language + hardware combinations appear in competition kernels
- how kernel-only competitions differ
- a high-level summary of key findings.

In [ ]:
# top language + accelerator combinations in competition kernels
comp_kv_full = comp_kernel_versions.merge(
    kernel_languages, left_on='ScriptLanguageId', right_on='Id', suffixes=('', '_lang'))
comp_kv_full = comp_kv_full.merge(
    accelerator_types, left_on='AcceleratorTypeId', right_on='Id', suffixes=('', '_accel'))

lang_accel = comp_kv_full.groupby(['DisplayName', 'Label']).size().reset_index(name='Count')
lang_accel = lang_accel.sort_values('Count', ascending=False).head(15)
lang_accel['Combo'] = lang_accel['DisplayName'] + ' + ' + lang_accel['Label']

fig = px.bar(lang_accel, x='Count', y='Combo', orientation='h',
             title='Top 15 Language + Accelerator Combinations in Competition Kernels',
             labels={'Combo': 'Language + Accelerator'},
             color_discrete_sequence=['#6C5B7B'])
fig.update_layout(width=1000, height=500, title_x=0.5,
                  yaxis={'categoryorder': 'total ascending'})
fig.show()

In [ ]:
# kernel-only competitions vs. open-submission competitions
kernel_only = competitions[competitions['OnlyAllowKernelSubmissions'] == True]
non_kernel_only = competitions[competitions['OnlyAllowKernelSubmissions'] == False]

print(f"Kernel-only competitions: {len(kernel_only):,}")
print(f"Open-submission competitions: {len(non_kernel_only):,}")
print()
print("=== Kernel-Only Competition Stats ===")
print(kernel_only[['TotalTeams', 'TotalCompetitors', 'TotalSubmissions']].describe().round(1))
print()
print("=== Open-Submission Competition Stats ===")
print(non_kernel_only[['TotalTeams', 'TotalCompetitors', 'TotalSubmissions']].describe().round(1))

#### 3.E.iii) ***Do Kernel-Only Competitions Level the Playing Field for Amateurs?***

Kernel-only competitions force all participants onto Kaggle's provided hardware, eliminating the hardware advantage that professionals may have. Do amateurs medal at higher rates in these competitions compared to open-submission ones?

In [ ]:
# Compare amateur medal rates: kernel-only vs. open-submission competitions
teams_konly = teams.merge(
    competitions[['Id', 'OnlyAllowKernelSubmissions']],
    left_on='CompetitionId', right_on='Id', suffixes=('', '_comp'))
teams_konly = teams_konly.merge(
    users[['Id', 'PerformanceTier']], left_on='TeamLeaderId', right_on='Id', suffixes=('', '_user'))

teams_konly['HasMedal'] = teams_konly['Medal'].notna() & (teams_konly['Medal'] != '')
teams_konly['CompType'] = teams_konly['OnlyAllowKernelSubmissions'].map(
    {True: 'Kernel-Only', False: 'Open Submission'})

tier_labels_full = {0: 'Novice', 1: 'Contributor', 2: 'Expert', 3: 'Master', 4: 'Grandmaster'}
teams_konly['TierLabel'] = teams_konly['PerformanceTier'].map(tier_labels_full)
teams_konly = teams_konly[teams_konly['TierLabel'].notna()]  # exclude Staff

# Medal rate by tier and competition type
medal_by_type_tier = teams_konly.groupby(['CompType', 'TierLabel']).agg(
    Total=('HasMedal', 'count'),
    Medals=('HasMedal', 'sum')
).reset_index()
medal_by_type_tier['MedalRate'] = (medal_by_type_tier['Medals'] / medal_by_type_tier['Total'] * 100).round(3)

tier_order = ['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster']

fig = px.bar(medal_by_type_tier, x='TierLabel', y='MedalRate', color='CompType',
             barmode='group',
             title='Medal Rate by Experience Level: Kernel-Only vs. Open Submission',
             labels={'TierLabel': 'Performance Tier', 'MedalRate': 'Medal Rate (%)', 'CompType': 'Competition Type'},
             category_orders={'TierLabel': tier_order},
             color_discrete_map={'Kernel-Only': '#20BEFF', 'Open Submission': '#F67280'})
fig.update_layout(width=1000, height=500, title_x=0.5)
fig.show()

# Print the key comparison for amateurs
print("Medal rates — Kernel-Only vs. Open Submission:\n")
for tier in tier_order:
    row_k = medal_by_type_tier[(medal_by_type_tier['TierLabel'] == tier) &
                                (medal_by_type_tier['CompType'] == 'Kernel-Only')]
    row_o = medal_by_type_tier[(medal_by_type_tier['TierLabel'] == tier) &
                                (medal_by_type_tier['CompType'] == 'Open Submission')]
    k_rate = row_k['MedalRate'].values[0] if len(row_k) > 0 else 0
    o_rate = row_o['MedalRate'].values[0] if len(row_o) > 0 else 0
    diff = k_rate - o_rate
    print(f"  {tier:15s}  Kernel-Only: {k_rate:.3f}%  |  Open: {o_rate:.3f}%  |  Diff: {diff:+.3f}%")

### 3.F) EDA Summary

In [ ]:
# ── EDA SUMMARY ──
print("=" * 70)
print("EXPLORATORY DATA ANALYSIS — KEY FINDINGS SUMMARY")
print("=" * 70)

# 1. Languages
total_kvs = len(kernel_versions)
python_ids = kernel_languages[kernel_languages['DisplayName'] == 'Python']['Id'].tolist()
python_kvs = len(kernel_versions[kernel_versions['ScriptLanguageId'].isin(python_ids)])
print(f"\n1. LANGUAGES (Research Q1):")
print(f"   Python (scripts + notebooks): {python_kvs:,} / {total_kvs:,} ({python_kvs/total_kvs*100:.1f}%)")
r_ids = kernel_languages[kernel_languages['DisplayName'] == 'R']['Id'].tolist()
r_kvs = len(kernel_versions[kernel_versions['ScriptLanguageId'].isin(r_ids)])
print(f"   R (scripts + notebooks):      {r_kvs:,} / {total_kvs:,} ({r_kvs/total_kvs*100:.1f}%)")

# 2. Solo vs team
total_teams_count = len(teams_with_size)
solo_count = int(teams_with_size['IsSolo'].sum())
print(f"\n2. INDIVIDUALS vs TEAMS (Research Q3):")
print(f"   Solo competitors: {solo_count:,} / {total_teams_count:,} ({solo_count/total_teams_count*100:.1f}%)")

# 3. Hardware
total_with_accel = int((kernel_versions['AcceleratorTypeId'] != 0).sum())
print(f"\n3. HARDWARE (Research Q5):")
print(f"   Kernels using any accelerator: {total_with_accel:,} / {total_kvs:,} ({total_with_accel/total_kvs*100:.1f}%)")
print(f"   Kernels with NO accelerator:   {total_kvs - total_with_accel:,} / {total_kvs:,} ({(total_kvs - total_with_accel)/total_kvs*100:.1f}%)")

# 4. Experience levels
total_users = len(users)
novice_count = int((users['PerformanceTier'] == 0).sum())
gm_count = int((users['PerformanceTier'] == 4).sum())
master_count = int((users['PerformanceTier'] == 3).sum())
print(f"\n4. EXPERIENCE LEVELS (Research Q4):")
print(f"   Novice users:   {novice_count:>10,} / {total_users:,} ({novice_count/total_users*100:.1f}%)")
print(f"   Grandmasters:   {gm_count:>10,} / {total_users:,} ({gm_count/total_users*100:.1f}%)")
print(f"   Masters:        {master_count:>10,} / {total_users:,} ({master_count/total_users*100:.1f}%)")

print("\n" + "=" * 70)